In [5]:
data_set = [[1.0, 2.1],
            [2, 1.1],
           [1.3,1.0],
           [1.0,1.0],
           [2.0,1.0]]
label = [1.0, 1.0, -1.0, -1.0, 1.0]
print data_set
print label

[[1.0, 2.1], [2, 1.1], [1.3, 1.0], [1.0, 1.0], [2.0, 1.0]]
[1.0, 1.0, -1.0, -1.0, 1.0]


In [98]:
import numpy as np
def predict_label(data_matrix, col_index, thresh_val,operator):
    result = np.ones(((np.shape(data_matrix)[0]),1))
    # <
    if operator == 'lt':
        result[data_matrix[:,col_index] <= thresh_val] = -1.0
    else:
        result[data_matrix[:,col_index ]> thresh_val] = -1.0
    return result


In [118]:
def create_single_adaboost(data_set, label,alpha):
    data_matrix = np.mat(data_set)
    label_vec = (np.mat(label)).T
    row_num, col_num = data_matrix.shape
  
    iteration_num = 10
    min_error = np.inf
    best_result= {}
    best_estimation = np.mat(np.zeros((row_num, 1)))
    
    for i in range(col_num):
        min_value = (data_matrix[:,i]).min()
        max_value = (data_matrix[:,i]).max()
        step_size = (max_value - min_value) / iteration_num
        
        for j in range(-1, iteration_num + 1):
            for op in ['lt', 'gt']:
                thresh_value = (min_value + float(j)*step_size)
                predict_result = predict_label(data_matrix, i, thresh_value, op)
                #print 'predict_result:  ', predict_result
                error = np.mat(np.ones((row_num,1)))
                error[predict_result == label_vec] = 0
                result_error = alpha.T * error
                #print 'result_error:  ',result_error
                if result_error < min_error:
                    min_error = result_error
                    best_result['best_col'] = i
                    best_result['thresh'] = thresh_value
                    best_result['op'] = op
                    best_estimation = predict_result.copy()
        return min_error, best_result, best_estimation
    
    
alpha = np.mat(np.ones((5,1)) / 5)


#min_error, best_result, best_estimation = create_single_adaboost(data_set, label, alpha)
#print best_estimation
#print best_result

In [119]:
def buildStump(dataArr,classLabels,D):
    dataMatrix = np.mat(dataArr); labelMat = np.mat(classLabels).T
    m,n = np.shape(dataMatrix)
    numSteps = 10.0; bestStump = {}; bestClasEst = np.mat(np.zeros((m,1)))
    minError = np.inf #init error sum, to +infinity
    for i in range(n):#loop over all dimensions
        rangeMin = dataMatrix[:,i].min(); rangeMax = dataMatrix[:,i].max();
        stepSize = (rangeMax-rangeMin)/numSteps
        for j in range(-1,int(numSteps)+1):#loop over all range in current dimension
            for inequal in ['lt', 'gt']: #go over less than and greater than
                threshVal = (rangeMin + float(j) * stepSize)
                predictedVals = predict_label(dataMatrix,i,threshVal,inequal)#call stump classify with i, j, lessThan
                errArr = np.mat(np.ones((m,1)))
                errArr[predictedVals == labelMat] = 0
                weightedError = D.T*errArr  #calc total error multiplied by D
                #print "split: dim %d, thresh %.2f, thresh ineqal: %s, the weighted error is %.3f" % (i, threshVal, inequal, weightedError)
                if weightedError < minError:
                    minError = weightedError
                    bestClasEst = predictedVals.copy()
                    bestStump['dim'] = i
                    bestStump['thresh'] = threshVal
                    bestStump['ineq'] = inequal
    return bestStump,minError,bestClasEst


In [125]:
def adaboost(data_set, label, iteration):
    weak_class = []
    data_mat = np.mat(data_set)
    row_num, col_num = np.shape(data_mat)
    weight_v = np.mat(np.ones((row_num,1)) / row_num)
    agg_class_predict = np.mat(np.zeros((row_num, 1)))
    for i in range(iteration):
        best_result,min_error,res_predict = buildStump(data_set, label, weight_v)
        print 'weight_v: ', weight_v
 
        alpha = float( 0.5 * np.log((1.0 - min_error) / max(min_error, 1e-6)))
        best_result['alpha'] = alpha
        weak_class.append(best_result)
        print 'res_predict: ', res_predict.T
        a = np.multiply(-1*alpha*(np.mat(label)).T, res_predict)
        
        weight_v = np.multiply(weight_v,np.exp(a))
        weight_v = weight_v / weight_v.sum()
        
        agg_class_predict += alpha * res_predict
        print 'sign_agg_class_predict: ', agg_class_predict
        print 'label: ', label
        
        agg_errors = 0.0
        for m in range(row_num):
            if np.sign(agg_class_predict)[m] != np.mat(label).T[m]:
                agg_errors += 1
        
        #agg_errors = np.multiply(np.sign(agg_class_predict) != np.mat(label).T, np.ones((row_num,1)))
        print 'agg_errors: ', agg_errors
        error_rate = agg_errors / row_num
        print 'error_rate: ', error_rate
        print '-----------------------------------------------\n'
        
        if error_rate == 0.0: break
    return weak_class
classify_arr = adaboost(data_set,label,9)
print 
print classify_arr[-1]0 

weight_v:  [[ 0.2]
 [ 0.2]
 [ 0.2]
 [ 0.2]
 [ 0.2]]
res_predict:  [[-1.  1. -1. -1.  1.]]
sign_agg_class_predict:  [[-0.69314718]
 [ 0.69314718]
 [-0.69314718]
 [-0.69314718]
 [ 0.69314718]]
label:  [1.0, 1.0, -1.0, -1.0, 1.0]
agg_errors:  1.0
error_rate:  0.2
-----------------------------------------------

weight_v:  [[ 0.5  ]
 [ 0.125]
 [ 0.125]
 [ 0.125]
 [ 0.125]]
res_predict:  [[ 1.  1. -1. -1. -1.]]
sign_agg_class_predict:  [[ 0.27980789]
 [ 1.66610226]
 [-1.66610226]
 [-1.66610226]
 [-0.27980789]]
label:  [1.0, 1.0, -1.0, -1.0, 1.0]
agg_errors:  1.0
error_rate:  0.2
-----------------------------------------------

weight_v:  [[ 0.28571429]
 [ 0.07142857]
 [ 0.07142857]
 [ 0.07142857]
 [ 0.5       ]]
res_predict:  [[ 1.  1.  1.  1.  1.]]
sign_agg_class_predict:  [[ 1.17568763]
 [ 2.56198199]
 [-0.77022252]
 [-0.77022252]
 [ 0.61607184]]
label:  [1.0, 1.0, -1.0, -1.0, 1.0]
agg_errors:  0.0
error_rate:  0.0
-----------------------------------------------


{'dim': 0, 'ineq': 'lt',

In [117]:
weight = np.mat([[0.5], [0.125],[0.125],[0.125],[0.125]])
print np.shape(weight)
create_single_adaboost(data_set, label, weight)

(5L, 1L)


(matrix([[ 0.25]]),
 {'best_col': 0, 'op': 'lt', 'thresh': 0.90000000000000002},
 array([[ 1.],
        [ 1.],
        [ 1.],
        [ 1.],
        [ 1.]]))

In [113]:
'''
def buildStump(dataArr,classLabels,D):
    dataMatrix = np.mat(dataArr); labelMat = np.mat(classLabels).T
    m,n = np.shape(dataMatrix)
    numSteps = 10.0; bestStump = {}; bestClasEst = np.mat(np.zeros((m,1)))
    minError = np.inf #init error sum, to +infinity
    for i in range(n):#loop over all dimensions
        rangeMin = dataMatrix[:,i].min(); rangeMax = dataMatrix[:,i].max();
        stepSize = (rangeMax-rangeMin)/numSteps
        for j in range(-1,int(numSteps)+1):#loop over all range in current dimension
            for inequal in ['lt', 'gt']: #go over less than and greater than
                threshVal = (rangeMin + float(j) * stepSize)
                predictedVals = predict_label(dataMatrix,i,threshVal,inequal)#call stump classify with i, j, lessThan
                errArr = np.mat(np.ones((m,1)))
                errArr[predictedVals == labelMat] = 0
                weightedError = D.T*errArr  #calc total error multiplied by D
                #print "split: dim %d, thresh %.2f, thresh ineqal: %s, the weighted error is %.3f" % (i, threshVal, inequal, weightedError)
                if weightedError < minError:
                    minError = weightedError
                    bestClasEst = predictedVals.copy()
                    bestStump['dim'] = i
                    bestStump['thresh'] = threshVal
                    bestStump['ineq'] = inequal
    return bestStump,minError,bestClasEst
weight = np.mat([[0.5], [0.125],[0.125],[0.125],[0.125]])
print buildStump(data_set, label, weight)
'''

({'dim': 1, 'ineq': 'lt', 'thresh': 1.0}, matrix([[ 0.125]]), array([[ 1.],
       [ 1.],
       [-1.],
       [-1.],
       [-1.]]))


In [133]:
def adaBoost_classify(test_data, classify_arr):
    test_data_matrix = np.mat(test_data)
    row_of_test_data = np.shape(test_data_matrix)[0]
    agg_predictor = np.mat(np.zeros((row_of_test_data,1)))
    
    for i in range(len(classify_arr)):
        res = predict_label(test_data_matrix,classify_arr[i]['dim'],classify_arr[i]['thresh'], classify_arr[i]['ineq'])
        agg_predictor += res* (classify_arr[i]['alpha'])
    return np.sign(agg_predictor)

#print classify_arr
print adaBoost_classify([0,0], classify_arr)

[{'dim': 0, 'ineq': 'lt', 'thresh': 1.3, 'alpha': 0.6931471805599453}, {'dim': 1, 'ineq': 'lt', 'thresh': 1.0, 'alpha': 0.9729550745276565}, {'dim': 0, 'ineq': 'lt', 'thresh': 0.90000000000000002, 'alpha': 0.8958797346140273}]
[[-1.]]


In [153]:
def get_data_from_file(filename):
    f = open(filename)
    data_set  = []
    label = []
    for line in f.readlines():
        string = line.strip().split('\t')
        tem_array = []
        for i in range(len(string) - 1):
            tem_array.append(float(string[i]))
        data_set.append(tem_array)
        label.append(float(string[len(string) - 1]))
    return data_set, label
train_data, train_label = get_data_from_file('horseColicTraining2.txt')
test_data, test_label = get_data_from_file('horseColicTest2.txt')
print train_label

[-1.0, -1.0, 1.0, -1.0, -1.0, 1.0, 1.0, -1.0, -1.0, 1.0, 1.0, 1.0, 1.0, -1.0, 1.0, -1.0, -1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, -1.0, 1.0, -1.0, -1.0, 1.0, 1.0, 1.0, -1.0, -1.0, 1.0, 1.0, -1.0, -1.0, 1.0, 1.0, -1.0, -1.0, -1.0, 1.0, 1.0, -1.0, -1.0, 1.0, -1.0, 1.0, 1.0, -1.0, -1.0, 1.0, -1.0, -1.0, -1.0, 1.0, 1.0, -1.0, -1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, -1.0, 1.0, -1.0, -1.0, -1.0, -1.0, 1.0, 1.0, -1.0, 1.0, -1.0, 1.0, 1.0, -1.0, 1.0, 1.0, -1.0, -1.0, 1.0, 1.0, 1.0, -1.0, 1.0, -1.0, 1.0, 1.0, 1.0, -1.0, 1.0, 1.0, 1.0, -1.0, 1.0, 1.0, -1.0, 1.0, 1.0, 1.0, -1.0, -1.0, -1.0, 1.0, 1.0, -1.0, 1.0, 1.0, 1.0, -1.0, 1.0, 1.0, 1.0, -1.0, 1.0, -1.0, -1.0, 1.0, 1.0, 1.0, -1.0, 1.0, -1.0, -1.0, 1.0, -1.0, -1.0, -1.0, 1.0, 1.0, 1.0, 1.0, -1.0, -1.0, 1.0, -1.0, 1.0, 1.0, -1.0, 1.0, 1.0, 1.0, -1.0, 1.0, 1.0, 1.0, 1.0, 1.0, -1.0, 1.0, 1.0, 1.0, -1.0, 1.0, -1.0, 1.0, -1.0, 1.0, 1.0, 1.0, -1.0, 1.0, -1.0, 1.0, -1.0, -1.0, 1.0, 1.0, -1.0, 1.0, -1.0, -1.0, 1.0, -1.0, 1.0, 1.0, 1.0,

In [154]:
horse_classify_arr = adaboost(train_data, train_label, 10)


weight_v:  [[ 0.00334448]
 [ 0.00334448]
 [ 0.00334448]
 [ 0.00334448]
 [ 0.00334448]
 [ 0.00334448]
 [ 0.00334448]
 [ 0.00334448]
 [ 0.00334448]
 [ 0.00334448]
 [ 0.00334448]
 [ 0.00334448]
 [ 0.00334448]
 [ 0.00334448]
 [ 0.00334448]
 [ 0.00334448]
 [ 0.00334448]
 [ 0.00334448]
 [ 0.00334448]
 [ 0.00334448]
 [ 0.00334448]
 [ 0.00334448]
 [ 0.00334448]
 [ 0.00334448]
 [ 0.00334448]
 [ 0.00334448]
 [ 0.00334448]
 [ 0.00334448]
 [ 0.00334448]
 [ 0.00334448]
 [ 0.00334448]
 [ 0.00334448]
 [ 0.00334448]
 [ 0.00334448]
 [ 0.00334448]
 [ 0.00334448]
 [ 0.00334448]
 [ 0.00334448]
 [ 0.00334448]
 [ 0.00334448]
 [ 0.00334448]
 [ 0.00334448]
 [ 0.00334448]
 [ 0.00334448]
 [ 0.00334448]
 [ 0.00334448]
 [ 0.00334448]
 [ 0.00334448]
 [ 0.00334448]
 [ 0.00334448]
 [ 0.00334448]
 [ 0.00334448]
 [ 0.00334448]
 [ 0.00334448]
 [ 0.00334448]
 [ 0.00334448]
 [ 0.00334448]
 [ 0.00334448]
 [ 0.00334448]
 [ 0.00334448]
 [ 0.00334448]
 [ 0.00334448]
 [ 0.00334448]
 [ 0.00334448]
 [ 0.00334448]
 [ 0.00334448]

weight_v:  [[ 0.00213157]
 [ 0.00855163]
 [ 0.00181817]
 [ 0.00536653]
 [ 0.0028726 ]
 [ 0.00181817]
 [ 0.00181817]
 [ 0.00536653]
 [ 0.00213157]
 [ 0.00812362]
 [ 0.00322667]
 [ 0.00322667]
 [ 0.00181817]
 [ 0.00536653]
 [ 0.00202488]
 [ 0.00213157]
 [ 0.00114098]
 [ 0.00181817]
 [ 0.00322667]
 [ 0.00602801]
 [ 0.00812362]
 [ 0.00181817]
 [ 0.00181817]
 [ 0.00202488]
 [ 0.00202488]
 [ 0.00322667]
 [ 0.00322667]
 [ 0.00181817]
 [ 0.00952387]
 [ 0.00114098]
 [ 0.00114098]
 [ 0.00536653]
 [ 0.00322667]
 [ 0.00322667]
 [ 0.00812362]
 [ 0.00114098]
 [ 0.00536653]
 [ 0.00952387]
 [ 0.00181817]
 [ 0.00536653]
 [ 0.00114098]
 [ 0.00322667]
 [ 0.00322667]
 [ 0.00339667]
 [ 0.00181817]
 [ 0.00114098]
 [ 0.01517639]
 [ 0.00181817]
 [ 0.0028726 ]
 [ 0.00114098]
 [ 0.00181817]
 [ 0.00536653]
 [ 0.00181817]
 [ 0.00181817]
 [ 0.0028726 ]
 [ 0.00536653]
 [ 0.00181817]
 [ 0.00213157]
 [ 0.00202488]
 [ 0.00114098]
 [ 0.00181817]
 [ 0.00322667]
 [ 0.0028726 ]
 [ 0.00457752]
 [ 0.00322667]
 [ 0.00202488]

weight_v:  [[ 0.00229314]
 [ 0.0098348 ]
 [ 0.00178491]
 [ 0.0042335 ]
 [ 0.0022232 ]
 [ 0.00131629]
 [ 0.00130885]
 [ 0.00782869]
 [ 0.00228018]
 [ 0.00431263]
 [ 0.00340552]
 [ 0.00316764]
 [ 0.00096522]
 [ 0.008369  ]
 [ 0.00156712]
 [ 0.00228018]
 [ 0.00122053]
 [ 0.00194493]
 [ 0.00316764]
 [ 0.00862712]
 [ 0.00588122]
 [ 0.00096522]
 [ 0.00178491]
 [ 0.00145766]
 [ 0.00213712]
 [ 0.00345163]
 [ 0.00340552]
 [ 0.0019081 ]
 [ 0.01005176]
 [ 0.00082136]
 [ 0.00122053]
 [ 0.0042335 ]
 [ 0.00171296]
 [ 0.00232279]
 [ 0.0085739 ]
 [ 0.00090009]
 [ 0.00415333]
 [ 0.00685597]
 [ 0.00130885]
 [ 0.00566399]
 [ 0.00090009]
 [ 0.00345163]
 [ 0.00461792]
 [ 0.00267954]
 [ 0.00194493]
 [ 0.00090009]
 [ 0.01601759]
 [ 0.00130885]
 [ 0.00307287]
 [ 0.00090009]
 [ 0.00178491]
 [ 0.00617178]
 [ 0.00178491]
 [ 0.00194493]
 [ 0.00419054]
 [ 0.00574068]
 [ 0.00194493]
 [ 0.00332414]
 [ 0.00216605]
 [ 0.00090009]
 [ 0.00178491]
 [ 0.00171296]
 [ 0.00301468]
 [ 0.00361107]
 [ 0.00461792]
 [ 0.00289795]

In [156]:
horse_predictor = adaBoost_classify(test_data, horse_classify_arr) 
error_count = 0.0
for i in range(len(horse_predictor)):
    if horse_predictor[i] != test_label[i]:
        error_count += 1.0
print float(error_count / len(horse_predictor))   

0.238805970149
